# Designing recursive functions

Now that we have seen how recursion works, we will focus on how to design recursive functions starting from an analysis of a problem. We will then translate the recursive functions in Python.

## General strategy
When confronted with a difficult problem, it is always helpful to draw it. The ability to draw something out is not just useful for aesthetic reasons: drawing the problem (with precision!) forces us to reckon with all the details of the problem, and to visualise them so we can properly deal with them.

After drawing or plotting the problem, we look for _regularities_. Regularities are powerful elements in a solution, because they suggest the building blocks of both the problem and therefore the code that will become the solution. Such regularities present themselves as smaller instances of the problem itself, which we also call _recurring structure_ or _inductive case_. Without regularities (or without seeing them) it is very hard to produce a solution.

After we have found the recurring structure, we must find what ties them together. For example, even though it is true that a line is made up of smaller lines, we must also define what "made up of" really means: in this case, concatenation.

The recurring structure should recur in a specific way: every time, at a smaller scale. The line must be made up of _smaller_ lines, and at some point we expect a line so small that we do not really need to split it further. This definition of when do we stop splitting our problem into smaller versions of itself is called the _base case_.

## Playing with numbers

As a first example, let us consider the problem of computing the sum of all numbers in the range $1 \dots n$ for a given $n$. Let us begin with drawing the sum of numbers:

[[PIC]]

The recurring structure can be seen just by putting brackets to identify that a sum is actually a smaller sum, plus the last number:

$$(0 + 1 + 2 + \dots + n-1 + n) = ((\dots(0 + 1) + 2)\dots) + n-1) + n$$

The brackets explicitly denote a sub-problem. Of course, we could rephrase this in a way that makes it even more evident where the recurring structure occurs. Whenever we have a series of numbers, such as $1 + \dots + n$ added together and within brackets, we will write $\text{sum}(1,n)$:

\begin{align*}
\text{sum}(1,n) 
&= \text{sum}(1,n-1) + n \\
&= (\text{sum}(1,n-2) + n-1) + n \\
&= ((\text{sum}(1,n-3) + n-2) + n-1) + n \\
&= \vdots \\
&= (((\dots(0 + 1 \dots) + n-2) + n-1) + n) \\
\end{align*}

This formulation should also make it evident how the base case will be zero: adding all numbers up to zero will simply result in zero itself.

We can translate this into a recursive function as follows:

In [6]:
def sum(n):
    if n <= 0: return 0
    else: return sum(n-1) + n
print(sum(int(input())))

6
21


We could of course also look for other recurring structures. For example, we could notice that the sum of numbers up until $n$ is actually the sum of all numbers within the range $1 \dots n$. If we focus on the sum over a range, then more decomposition possibilities emerge. One of these would, for example, split the range in two half ranges:

$0 + 1 + \dots + n = (0 + 1 + \dots + n/2-1) + (n/2 + n/2+1 + \dots + n)$$

For example, the sum $1 + 2 + 3 + 4$ could be reformulated as the sum of $1 + 2$ and $3 + 4$. The sub-sequences respectively end at the middle point of the previous subsequence, $2 = (1+4)//2$, and start at its successor $3$.

The subsequences we add according to this formulation do not vary only in the last number to add, but also the beginning of the range. We will denote the beginning of the sequence with $l$ (_lower_ bound) and the end of the sequence with $u$ (_upper_ bound).

When adding (sub-) sequences, we actually have two base cases: the empty sequence, when $l > u$ (it is empty because there can be no values between numbers where the first is greater than the second), adds to $0$; the sequence with only one element, when $l = u$, adds to $l$.

\begin{align*}
\text{sum_range}(l,u) &= 0 \text{ when } l > u \\
&= l \text{ when } l = u \\
&= \text{sum_range}(l,m) + \text{sum_range}(m+1,u) \text{ otherwise, where } m = (l+u)/2 \\
\end{align*}

Turned into code, this becomes:

In [15]:
def sum_range(l, u):
    if l > u:
        return 0
    elif l == u:
        return l
    else:
        middle_point = (l + u) // 2
        fst_half = sum_range(l, middle_point)
        snd_half = sum_range(middle_point + 1, u)
        return fst_half + snd_half

def sum(n):
    return sum_range(1, n)

print(sum(int(input())))

7
28


### More examples with numbers

We can perform a very similar operation to adding a range of numbers: multiplying a range. For example, the product of numbers up until $3$ would be $1 \times 2 \times 3$.

Just like we did for the sum, we can decompose the product into the product of a shorter range, multiplied by the last number. Thus, the product of an empty sequence of numbers is $\text{prod}(1,0) = 1$ (should we choose $0$, then we would always get $0$ also for non-empty sequences!), whereas the product of a non-empty sequence of numbers up to $n$ is the product of the sequence up to $n-1$, multiplied by $n$: $\text{prod}(1,n) = \text{prod}(1,n-1) \times n$.

This becomes the simple recursive function:

In [23]:
def prod(n):
    if n <= 0:
        return 1
    else:
        return prod(n-1) * n
print(prod(int(input())))

4
24


With the very same trick, we could define the power $b^a = \underbrace{b \times \dots \times b}_{a \text{ times}}$ as $1$ when $a=0$, but \underbrace{b \times \dots \times b}_{a-1 \text{ times}} \times b$ otherwise.

This directly translates into the (now familiar-looking) code:

In [26]:
def pow(b,a):
    if a <= 0:
        return 1
    else:
        return pow(b,a-1) * b
print(pow(int(input()), int(input())))

4
2
16


## Drawing recursively
- line
    - asterisk + line
    - line + asterisk + line
- rectangle
    - line + rectangle
    - rectangle + rectangle
    - rectangle + rectangle along the longest axis
- triangle
    - triangle + line
    - trapezoid + trapezoid